In [1]:
import matplotlib.pyplot as plt
import sklearn.metrics as skm

import docopt
import numpy as np
from Bio import SeqIO
import copy
from NNfxns import neural_network
from NNfxns import train


In [2]:
import numpy as np

"""
-Make a NN class
-define my functions
  -autoencoder and its parameters
  -initial values/input
  -vecotorization
  -expected values
  -output
  -backwards propagation
  -feedforward propagation
  -weights/bias
  -sigmoid function
  -sigmoid derivative

"""
# Make a class to allow self reference
class neural_network():

# The NN will have a number of input neurons, a hidden layer, and number of an output neurons of the specified numbers, 
# which we apply to the class and can adjust as necessary later

    def __init__(self, input_layer_nodes=8, hidden_layer_nodes=3, output_layer_nodes=8):
        self.input_layer_nodes = input_layer_nodes
        self.hidden_layer_nodes = hidden_layer_nodes
        self.output_layer_nodes = output_layer_nodes

        # We want the autoencoder to learn the best weights for the data so we 
        # initialize matrices with random weights 0.1 > x > -0.1
        # These will link the autoencoder's different layers.
        
        self.connection_matrix_1 = np.random.randn(self.input_layer_nodes, self.hidden_layer_nodes) / 10
        self.connection_matrix_2 = np.random.randn(self.hidden_layer_nodes, self.output_layer_nodes) / 10

        # We have several vectors that our encoder will make to take in (input) and put out (output) data.
        # Recall the hidden layer will reduce the dimension, in a vector-wise fashion
        
        self.input_vector = None
        self.hidden_layer_output = None
        self.output_layer_output = None

        # We can also start off our autoencoder with a biased vector or matrices, depending on what we need from our model
        
        self.input_with_bias = None
        self.matrix_1_bias = None
        self.matrix_2_bias = None

        # Recall that we need to take the derivative of the cost function with respect to weight, and develop matrices from this
        
        self.hidden_dx_matrix = np.zeros([self.hidden_layer_nodes, self.hidden_layer_nodes])
        self.output_dx_matrix = np.zeros([self.output_layer_nodes, self.output_layer_nodes])

        # Hyperparameter: Learning Rate (controls how quickly a NN learns a problem)
        # Typically in a range between [0.1, 1], configureable
        # A perfect learning rate will make the model learn to best approximate the function given available resources (layers/nodes)
        # in a given number of epochs
        # We'll start with a large one because we've got other stuff to do, dudes. The pitfall of that is that it will likely arrive at a suboptimal
        # set of weights.
        
        self.learning_rate = 1

        
        
        # Bit conversion for DNA into NN inputs because I'm too lazy to think of a different way to encode these. Recall, we want either 0 or 1, not anything continuous.
        
        self.base_binary_conversion = {'A': '0001',
                                       'C': '0010',
                                       'T': '0100',
                                       'G': '1000'
                                       }

        
        
        
        # These are the expected values for our NN to return
        
        self.expected_values = None



    # Now we make moves to fill in and interpret those matrices
    
    # Initialize the values of the bias matrix given the information passing through the hidden layer and output layer
    
    def initialize_values(self):
        bias_ones_1 = np.ones([1, self.hidden_layer_nodes])
        self.matrix_1_bias = np.append(self.connection_matrix_1, bias_ones_1, axis=0)

        bias_ones_2 = np.ones([1, self.output_layer_nodes])
        self.matrix_2_bias = np.append(self.connection_matrix_2, bias_ones_2, axis=0)

    # We'll make the binding sites into binary bits that the encoder can interpret, and then tell it what to expect 
    
    def set_input_and_expected_values(self, input_DNA, autoencoder=True, negative=True):
        # Convert input DNA sequence into binary bits
        self._construct_input_vector(input_DNA)
        # Set expected value depending on autoencoder or werk
        self._set_expected_values(autoencoder, negative)
        # Weight matrices and input/output vectors with bias applied
        self.input_with_bias = np.append(self.input_vector, [1])

    
    
    # This directly handles conversion of the DNA binding site string to the 1/0 vector describing it, and assigns it to the original input class
    
    def _construct_input_vector(self, input_DNA):
       
        temp_vector_list = []

        for base in input_DNA:
            for number in self.base_binary_conversion[base]:
                temp_vector_list.append(float(number))

        self.input_vector = np.asarray(temp_vector_list)

    
    
    # This will set the values we expect from the NN depending on whether or not we use the autoencoder (T/F).
    
    def _set_expected_values(self, autoencoder=True, negative=True):
        if autoencoder == True:
            self.expected_values = self.input_vector

        if autoencoder == False:
            if negative == True:
                self.expected_values = 0
            if negative == False:
                self.expected_values = 1

    # Recall an autoencoder is a feedforward method. We need to convert input to hidden layer reduced dim info to output layer
    # that is the same as the input.
    
    def forward_propogation(self):
        # Generates hidden layer outputs
        
        output_one_list = []

        for element in np.nditer(np.dot(self.input_with_bias, self.matrix_1_bias)):
            output_one_list.append(self._sigmoid_function(element))
        self.hidden_layer_output = np.asarray(output_one_list)

        # Calculate the square derivate matrix for the hidden layer outputs
        
        for position, value in enumerate(self.hidden_layer_output):
            self.hidden_dx_matrix[position][position] = self._sigmoid_function_derivative(value)

        # The results from the output layer 
        # Add bias to hidden_layer_output
        
        self.hidden_output_bias = np.append(self.hidden_layer_output, [1])

        output_two_list = []
        for element in np.nditer(np.dot(self.hidden_output_bias, self.matrix_2_bias)):
            output_two_list.append(self._sigmoid_function(element))
        self.output_layer_output = np.asarray(output_two_list)

        # Calculate square derivate matrix for output layer outputs
        
        for position, value in enumerate(self.output_layer_output):
            self.output_dx_matrix[position][position] = self._sigmoid_function_derivative(value)

    
    # Recall that in stochastic gradient descent, we estimate the error gradient for the current state of the model using
    # examples from the training. The backwards propagation is what we use to update the weights of the model. 
    
    def backward_propogation(self):
        # Output Layer error
        
        deviations = self.output_layer_output - self.expected_values

        output_layer_errors = np.dot(self.output_dx_matrix, deviations)

        # Hidden Layer error
        
        hidden_layer_errors = np.dot(np.dot(self.hidden_dx_matrix, self.connection_matrix_2), output_layer_errors)

        # Matrix 2 Errors (those of our output layer)
        
        output_rated_row_vector = np.asmatrix(-(self.learning_rate * output_layer_errors)).transpose()
        matrix_2_errors_transposed = np.dot(output_rated_row_vector, np.asmatrix(self.hidden_output_bias))
        self.matrix_2_errors = matrix_2_errors_transposed.transpose()

        # Matrix 1 Errors (those of our hidden layer)
        
        hidden_rated_row_vector = np.asmatrix(-(self.learning_rate * hidden_layer_errors)).transpose()
        matrix_1_errors_transposed = np.dot(hidden_rated_row_vector, np.asmatrix(self.input_with_bias))
        self.matrix_1_errors = matrix_1_errors_transposed.transpose()

    
    # Basically the sum between the output and hidden layers' bias and errors is what we use...
    
    def update_weights_and_bias(self):
        self.matrix_1_bias = self.matrix_1_bias + self.matrix_1_errors
        self.matrix_2_bias = self.matrix_2_bias + self.matrix_2_errors

        self.connection_matrix_1 = self.matrix_1_bias[:-1]
        self.connection_matrix_2 = self.matrix_2_bias[:-1]

    
    # The activation function for the layers. Some people use relu. Its more of less simple to take the deriv of sigmoid. 
    
    def _sigmoid_function(self, input):
        return float(1 / (1 + np.exp(-input)))

    
    # Recall we also take the derivative of the activatio function. 
    def _sigmoid_function_derivative(self, input):
        return float(self._sigmoid_function(input) * (1 - self._sigmoid_function(input)))




In [3]:
"""
Training the neural net

What can be used:
     autoencoder
     %%%%%%%%werk <partitions> <sampling>
     test
Arguments:
    autoencoder
        Run ze autoencoder
    %%%%%%%%werk
        Do the rap1 learning task including cross-validation
    test
        Classify test data and output to tsv file type
    <partitions>
        Number of partitions to make for cross-valitation (k-fold!)
    <sampling>
        Sampling method for NN training input
        (slide) Go over each sequence in 17 nucleotide sliding frame (the length of the positives/test binding sites)
        (space) Each sequence is cut into 17 nucleotide bits for inputs (the binary bits that are useable by our model)
"""

def werk():
    """
    Train neural network on RAP1 binding sites
        * Input layer with 17*4 nodes (because 17 different nucleotides defining each sequence, and 4 different
          possible nucleotides describing those positions) + bias
        * Hidden layer with 23-35 nodes (merge at least 2 input neurons)+ bias
        * One output layer node (number of neurons in the output layer will equal the number of outputs
          associated with each input; we want one answer)
    Train against negative and positive binding sites
        * Import all negative sequences from .fa file
        * For each sequence, iterate every 17 bases and train with
          expected of 0 (since these are negatives, while positives should be 1)
        * Because it is so important to have a 1:1 ratio of negatives:positives when training our model, 
          for every 137 negative training instances, need to train it against all positive binding sites
          with expected of 1 
        * Will continue until negative binding sites have been fully ran through
    """

    # This part takes care of bringing in all those positive sequences
   
    positive_sites = [pos_seq.strip() for pos_seq in open('data/rap1-lieb-positives.txt')]

    
    
    # This part takes care of bringing in all those neg sequences
    
    negative_sites = list(SeqIO.parse('data/yeast-upstream-1k-negative.fa', 'fasta'))
    
    

    # Separate into random sections for the k-fold x-validation for both our positives and negatives 
    # Taken from : http://stackoverflow.com/questions/3352737/python-randomly-partition-a-list-into-n-nearly-equal-parts
    
    
    partitions = int(58.8235294118)
    neg_division = len(negative_sites) / float(partitions) # how many partitions we can make from the sites given
    neg_randomly_partitioned_list = [negative_sites[int(round(neg_division * i)): int(round(neg_division * (i + 1)))]
                                     for i in range(partitions)] # makes a list of those partitions thru each site

    pos_division = len(positive_sites) / float(partitions) # ditto ^^
    pos_randomly_partitioned_list = [positive_sites[int(round(pos_division * i)): int(round(pos_division * (i + 1)))]
                                     for i in range(partitions)]

    
    
    # Go thru neg sites subsets for x-validation, keep track of how many separations we do based on partitions
    
    separation = 0
    for index in range(int(58.8235294118)):
        # Set up cross-validation sets for the positives and negatives
        neg_site_list_copy = copy.deepcopy(neg_randomly_partitioned_list)
        del neg_site_list_copy[index]
        neg_site_training = [seq for partition in neg_site_list_copy for seq in partition]
        neg_cross_validation_set = neg_randomly_partitioned_list[index]

        pos_site_list_copy = copy.deepcopy(pos_randomly_partitioned_list)
        del pos_site_list_copy[index]
        pos_site_training = [seq for partition in pos_site_list_copy for seq in partition]
        pos_cross_validation_set = pos_randomly_partitioned_list[index]

        print("Training on the training set...")

        # Input our hyperparameters = # nodes
        NN = neural_network(68, 23, 1)

        # See neural_net.py to get info on initialization
        NN.initialize_values()

        pos_counter = 0
        counter = 0

        # If we're sampling from tneg then we'll slide over 17 nucleotides 
        
        
        for site in neg_site_training:

                # Iterate over site in 17 nucleotide sliding frames in negative sites, decide which model to use
                for block in range(len(site) - 16):
                    slice = site[block:(block + 17)].seq
                    if slice not in positive_sites:
                        if all([slice[4] == 'C', slice[5] == 'C', slice[9] == 'C']) == False:
                            NN.set_input_and_expected_values(slice, autoencoder=False, negative=True)
                            NN.forward_propogation()
                            NN.backward_propogation()
                            NN.update_weights_and_bias()
                            pos_counter += 1
                        else:
                            print(slice)

                    if pos_counter == len(pos_site_training):
                        for pos_site in pos_site_training:
                            NN.set_input_and_expected_values(pos_site, autoencoder=False, negative=False)
                            NN.forward_propogation()
                            NN.backward_propogation()
                            NN.update_weights_and_bias()

                        pos_counter = 0

                # have reset the positives counter and will now say that we've done some training on those
                
                counter += 1

                print("Training set: {}/{} completed...".format(counter, len(neg_cross_validation_set)))

                max_change_1 = NN.matrix_1_errors.max()
                min_change_1 = NN.matrix_1_errors.min()
                max_change_2 = NN.matrix_2_errors.max()
                min_change_2 = NN.matrix_2_errors.min()

                if any([max_change_1 < 0.00000000001 and max_change_1 > 0,
                        min_change_1 > -.00000000001 and min_change_1 < 0]) and any(
                    [max_change_2 < 0.00000000001 and max_change_2 > 0,
                     min_change_2 > -0.00000000001 and min_change_2 < 0]):
                    print("Stop criterion met after {} iterations".format(counter))
                    break
        #when we sample from the negatives we only take 17 nucleotide chunks from each site
        for site in neg_site_training:
                
                number_of_blocks = int(len(site) / 17) #length of neg site tells us the amount of 17 length chunks possible

                for block in range(number_of_blocks):
                    slice = site[(block * 17):((block + 1) * 17)].seq
                    if slice not in positive_sites:
                        if all([slice[4] == 'C', slice[5] == 'C', slice[9] == 'C']) == False:
                            NN.set_input_and_expected_values(slice, autoencoder=False, negative=True)
                            NN.forward_propogation()
                            NN.backward_propogation()
                            NN.update_weights_and_bias()
                            pos_counter += 1

                        else:
                            print(slice)

                    #quick check to make sure that we've finished going thru the positives yet
                    if pos_counter == len(pos_site_training):
                        for pos_site in pos_site_training:
                            NN.set_input_and_expected_values(pos_site, autoencoder=False, negative=False)
                            NN.forward_propogation()
                            NN.backward_propogation()
                            NN.update_weights_and_bias()

                        pos_counter = 0

                    counter += 1

                max_change_1 = NN.matrix_1_errors.max()
                min_change_1 = NN.matrix_1_errors.min()
                max_change_2 = NN.matrix_2_errors.max()
                min_change_2 = NN.matrix_2_errors.min()

                if any([max_change_1 < 0.00000000001 and max_change_1 > 0,
                        min_change_1 > -.00000000001 and min_change_1 < 0]) and any(
                    [max_change_2 < 0.00000000001 and max_change_2 > 0,
                     min_change_2 > -0.00000000001 and min_change_2 < 0]):
                    print("Stop criterion met after {} iterations".format(counter))
                    break
        
            
        # taken each partition and trained model
        print("Performing Cross-validation")

        pos_list = []
        neg_list = []

        
        # Return the sets of positives and negatives from the x-validation
        
        print("Negative cross-validation set...")
        counter = 0
        for site in neg_cross_validation_set:
            for slice in range(len(site) - 16):
                NN.set_input_and_expected_values(site[slice:slice + 17].seq, autoencoder=False, negative=True)
                NN.forward_propogation()
                neg_list.append(NN.output_layer_output)
            counter += 1
            print("Negative cross-validation: {}/{} completed...".format(counter, len(neg_cross_validation_set)))
            break

        print("Positive cross-validation set...")
        for site in pos_cross_validation_set:
            NN.set_input_and_expected_values(site, autoencoder=False)
            NN.forward_propogation()
            pos_list.append(NN.output_layer_output)

        print('Positive avg: {}'.format(sum(pos_list) / len(pos_list)))
        print('Negative avg: {}'.format(sum(neg_list) / len(neg_list)))
        print(NN.matrix_1_bias)
        print(NN.matrix_2_bias)

        # Output the connection matrices with greatest separation between the average positive and negative scores
        if ((sum(pos_list) / len(pos_list)) - (sum(neg_list) / len(neg_list))) > separation:
            np.savetxt('connection_matrix_1.csv', NN.matrix_1_bias, delimiter=',')
            np.savetxt('connection_matrix_2.csv', NN.matrix_2_bias, delimiter=',')
            separation = (sum(pos_list) / len(pos_list)) - (sum(neg_list) / len(neg_list))


# A simple definition of the autoencoder that uses those same NN parameters
def autoencoder():
  
    NN = neural_network()
    NN.set_input_and_expected_values('GA', autoencoder=True)
    NN.initialize_values()

    # Stop criterion
    finished_working = False

    while finished_working == False:
        NN.forward_propogation()
        NN.backward_propogation()
        NN.update_weights_and_bias()

        max_change_1 = NN.matrix_1_errors.max()
        min_change_1 = NN.matrix_1_errors.min()
        max_change_2 = NN.matrix_2_errors.max()
        min_change_2 = NN.matrix_2_errors.min()

        if any([max_change_1 < 0.00001 and max_change_1 > 0,
                min_change_1 > -.00001 and min_change_1 < 0]) or any(
            [max_change_2 < 0.00001 and max_change_2 > 0,
             min_change_2 > -0.00001 and min_change_2 < 0]):
            finished_working = True

    print(NN.output_layer_output)

def test():
    test_sequences = open('data/rap1-lieb-test.txt')
    NN = neural_network(68, 23, 1)
    NN.matrix_1_bias = np.loadtxt('connection_matrix_1.csv', delimiter=',')
    NN.matrix_2_bias = np.loadtxt('connection_matrix_2.csv', delimiter=',')

    NN_outputs = open('NN_predictions.txt', 'w')

    for test_seq in test_sequences:
        NN.set_input_and_expected_values(test_seq.strip())
        NN.forward_propogation()
        NN_outputs.write('{}\t{}\n'.format(test_seq.strip(), NN.output_layer_output[0]))

    NN_outputs.close()

if __name__ == 'train':
    import docopt
    import numpy as np
    from Bio import SeqIO
    import copy
    from .NNfxns import neural_network

    args = docopt.docopt(__doc__)

    if args['autoencoder']:
        autoencoder()

    if args['werk']:
        werk()

    if args['test']:
        test()



In [4]:
werk()

Training on the training set...
ATTACCAGCCAATTCTA
ACCACCGGACATTATAT
AACACCTTACGCGTGAG
TGTGCCCACCGCTTCGC
Training set: 1/55 completed...
GTGACCTGACTGATTTA
ATATCCCTTCTTATGAA
GACCCCCTACTCATACT
CTGTCCTTTCTGTTCGG
Training set: 2/55 completed...
AAGTCCGAGCGGCAGAG
GTATCCAAGCCCAAAAG
TAGGCCATTCACTGCTG
TGTGCCACGCCAGCAGT
AAGCCCTGTCAAGTGTC
TCCACCGCACATTAGAA
TCTACCCTTCCAATCAC
CTACCCTTCCAATCACA
CCTTCCAATCACACATA
GCTTCCTTCCACTCATG
CCTTCCACTCATGCGAA
Training set: 3/55 completed...
Stop criterion met after 3 iterations
ACCACCGGACATTATAT
AACACCTTACGCGTGAG
Stop criterion met after 61 iterations
Performing Cross-validation
Negative cross-validation set...
Negative cross-validation: 1/55 completed...
Positive cross-validation set...
Positive avg: [0.9967672]
Negative avg: [0.03426321]
[[ 0.00692875 -0.10223647 -0.14198745 ...  0.05431373 -0.18778397
  -0.03944165]
 [-0.16256641 -0.47754481  0.41934637 ...  0.16644385  0.62020042
  -0.12097855]
 [-0.05771382 -0.24730907  0.20611787 ... -0.12534712  0.275817

In [5]:
connection_matrix_1 = np.loadtxt('connection_matrix_1.csv', delimiter=',')
connection_matrix_2 = np.loadtxt('connection_matrix_2.csv', delimiter=',')

In [6]:
print(connection_matrix_2)

[-2.07672766 -2.42082962 -2.16515683 -2.1817802  -0.89952326 -1.96869302
 -1.98680373 -2.0702102  -2.0510714  -2.19335789 -2.1407531   0.99484564
 -2.16514628  0.69044834 -2.31086253 -1.8399765   0.15736444  0.95652668
 -2.08289844 -1.21405261  0.97566657  1.81204494  1.58817711  0.2417018 ]


In [7]:
import pandas as pd
connection_matrix_1 = pd.DataFrame(connection_matrix_1)
connection_matrix_2 = pd.DataFrame(connection_matrix_2)



In [9]:
connection_matrix_1

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,-0.004650,0.317164,0.187555,0.036338,0.167651,0.171043,0.172984,0.195501,0.000773,0.147696,...,0.101433,0.163019,0.008907,0.212484,-0.220691,0.252872,-0.023810,-0.074828,-0.278806,-0.358458
1,0.219465,0.345566,0.369550,0.524495,0.004241,0.324165,0.391306,0.221453,0.056690,0.420052,...,-0.261208,0.236428,0.140516,-0.113943,0.103255,0.304786,-0.052655,-0.113552,-0.269752,-0.135084
2,0.096171,0.145501,0.142182,0.246359,-0.092733,0.165250,0.195279,0.139327,0.030534,0.394047,...,-0.136190,0.165312,0.069130,-0.104643,-0.198787,0.114394,0.020274,-0.125172,-0.257269,-0.331013
3,-0.387988,-0.281288,-0.417239,-0.366003,-0.103298,-0.133998,-0.477301,-0.473021,-0.086918,-0.475605,...,0.027270,-0.381525,0.020841,0.004041,0.072224,-0.550831,-0.122892,0.012376,0.071837,0.046551
4,0.466404,0.475845,0.256896,0.534455,-0.070794,0.287328,0.398532,0.398389,0.241146,0.459260,...,-0.253753,0.342251,0.247145,-0.067761,-0.281562,0.612792,-0.063629,-0.118593,-0.248041,-0.328949
5,0.529243,0.556358,0.530682,0.629652,0.020078,0.110972,0.776699,0.527862,0.246058,0.718878,...,-0.142497,0.641440,0.142143,-0.309513,-0.193206,0.663986,0.109818,-0.147500,-0.553786,-0.379398
6,-1.087535,-1.300430,-1.169769,-1.343545,-0.307972,-0.671947,-1.210566,-0.835221,-0.784098,-1.353855,...,0.091954,-1.207550,-0.529326,0.076526,0.141653,-1.263534,-0.273671,0.255178,0.507452,0.480117
7,0.342559,0.404173,0.210296,0.500334,0.154135,0.150600,0.563879,0.294450,0.158602,0.410509,...,-0.042432,0.399952,0.100276,-0.037050,-0.247425,0.197196,0.072059,0.025639,-0.295502,-0.146926
8,-0.047075,0.149444,0.048001,0.203117,0.007090,0.088329,0.267158,0.117350,-0.018401,0.067310,...,-0.229362,0.129283,-0.017043,-0.051010,-0.201680,0.042419,-0.090125,0.049805,-0.310643,-0.042181
9,0.522330,0.544399,0.653813,0.705769,0.280360,0.291754,0.521837,0.609323,0.524647,0.839235,...,0.043266,0.498099,0.248496,-0.024952,-0.116783,0.679499,0.210882,-0.454102,-0.569666,-0.665071


In [10]:
test()

In [11]:
ls

8by8.png                 README.md                setupnice/
8by8code.png             connection_matrix_1.csv  test/
Helperfunctions.ipynb    connection_matrix_2.csv  test_auto.png
NN_predictions.txt       data/
NNfxns/                  requirements.txt
